In [1]:
# Import local files
from manage_path import *
from get_data import *
from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
data = load_data(file_name="FINRA_TRACE_2014.pkl")

loading data FINRA_TRACE_2014.pkl...
Getting data from/home/raymond/Research/FINRA_TRACE/Data/Pickle/FINRA_TRACE_2014.pkl...
Data getting success!


In [3]:
data.head()

,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,ENTRD_VOL_QT,RPTD_PR,RPT_SIDE_CD,Year,TRD_EXCTN_DTTM,TRD_RPT_DTTM,Report_Dealer_Index,Contra_Party_Index,TRC_ST,RATING_MR,RATING_MR_Numeric
0,RBS3706239,00077TAA2,CORP,10000.0,111.245,D,2014.0,2014-01-31 10:40:23,2014-01-31 10:40:37,2048,2917,T,Ba1,11.0
1,RBS3706239,00077TAA2,CORP,2000.0,113.389,C,2014.0,2014-02-21 11:19:41,2014-02-21 11:19:57,2614,99999,T,Ba1,11.0
2,RBS3706239,00077TAA2,CORP,5000.0,112.070,C,2014.0,2014-01-24 15:19:50,2014-01-24 15:20:18,3513,99999,T,Ba1,11.0
3,RBS3706239,00077TAA2,CORP,2000.0,112.640,D,2014.0,2014-02-21 11:19:41,2014-02-21 11:20:01,2048,2514,T,Ba1,11.0
4,RBS3706239,00077TAA2,CORP,10000.0,112.245,D,2014.0,2014-01-16 13:41:11,2014-01-16 13:41:27,2048,2940,T,Ba1,11.0


In [4]:
data['price'] = (data['ENTRD_VOL_QT'] * data['RPTD_PR'])/100

In [5]:
data['price'].describe()

count    6.323262e+06
mean     6.073740e+05
std      2.391643e+06
min      0.000000e+00
25%      1.103970e+04
50%      3.379530e+04
75%      2.457833e+05
max      1.154484e+09
Name: price, dtype: float64

In [6]:
"""Compute Dc_v4 which is count of bonds on given dealer and day seperated buy and sell"""
data['document_date'] = data['TRD_EXCTN_DTTM'].dt.date.apply(lambda x: str(x))
create_buy_document_no_source_vectorize = np.vectorize(create_buy_document_no_source)
create_sell_document_no_source_vectorize = np.vectorize(create_sell_document_no_source)
client_to_delete_vectorize = np.vectorize(client_to_delete)
print("creating documents ......")
# Add new column Dc_v4_S which is the string representation of report dealer buy on the specific day
data['Dc_v4_S'] = create_sell_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
# Add new column Dc_v4_B which is the string representation of report dealer sell on the specific day
data['Dc_v4_B'] = create_buy_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
print("documents created!!")

creating documents ......
documents created!!


In [13]:
type(data['Dc_v4_S'].iloc[7])

str

In [14]:
1 != 2

True

In [18]:
data_sub = data[['Dc_v4_S','Dc_v4_B','BOND_SYM_ID','price']].copy()
data_gb_sell = data_sub[data_sub['Dc_v4_S']!='nan'].groupby(by=['Dc_v4_S','BOND_SYM_ID'])
data_gb_buy = data_sub[data_sub['Dc_v4_B']!='nan'].groupby(by=['Dc_v4_B','BOND_SYM_ID'])

In [19]:
#sell_matrix = data_gb_sell['price'].sum().unstack(fill_value=0)

In [25]:
sell_df = data_gb_sell['price'].sum().to_frame()

In [26]:
bins=[0,10000,100000,1000000,10000000,sell_df[sell_df['price']>0]['price'].max()]
labels = [i for i in range(len(bins)-1)]
pd.cut(sell_df[sell_df['price']>0]['price'],bins=bins,labels=labels)

Dc_v4_S             BOND_SYM_ID
10,2014-01-07,StC   ONEX3691468    3
10,2014-01-09,StD   ONEX3691468    2
10,2014-01-13,StC   WAC4080776     2
10,2014-01-14,StC   TC.AC          2
10,2014-01-14,StD   TC.AC          2
10,2014-01-21,StD   CRBM3693244    2
10,2014-01-22,StC   BUFF.GA        3
                    CRBM3693244    3
10,2014-01-30,StC   EVOR3968195    3
10,2014-02-06,StC   TSG3851467     3
10,2014-02-07,StC   ONEX3691468    3
10,2014-02-07,StD   BAIN3685563    2
10,2014-02-11,StC   ROIA.GF        3
10,2014-02-18,StD   RCII.GH        3
10,2014-02-19,StD   RCII.GH        2
10,2014-02-27,StC   CCMO4094541    3
10,2014-03-06,StC   PUCR3852347    2
10,2014-03-06,StD   CDFA.GC        2
10,2014-03-07,StC   ANR.GC         3
                    AONS.GB        1
10,2014-03-10,StD   CDFA.GC        2
10,2014-03-11,StC   MPM3672628     2
10,2014-03-12,StC   ATPG.GE        1
10,2014-03-12,StD   CDFA.GC        2
10,2014-04-01,StD   BAIN3685563    2
10,2014-04-07,StC   PUCR3852347    2
10,201

In [27]:
pd.DataFrame(preprocessing.minmax_scale(sell_df[sell_df['price']>0],feature_range=(0, 5))).describe()

,0
count,3.458211e+06
mean,2.651762e-03
std,9.961698e-03
min,0.000000e+00
25%,6.066659e-05
50%,2.039351e-04
75%,1.211984e-03
max,5.000000e+00


In [ ]:
sell_df.groupby(level=0).count().iloc[12::].describe()

In [ ]:
data_gb_sell['price'].sum().describe()

In [ ]:
from scipy.stats import mstats

In [ ]:
data['price_winsor'] = pd.Series(mstats.winsorize(data['price'].values,limits=[0.10,0.10]))

In [ ]:
pd.Series(preprocessing.minmax_scale(data['price_winsor'],feature_range=[0,5])).describe()

In [ ]:
data['price_winsor']